In [ ]:
from process import process, status
from transforms import master_transform
import pandas as pd
import matplotlib
float_format = '{:,.2f}'
pd.options.display.float_format = float_format.format
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import datetime

In [ ]:
mt = master_transform.Loader()
pr = process.Process()
df = pr.process(mt.load_all())

In [ ]:
outstanding_mortgage = df[df.Account == 'Lloyds Mortgage'].Amount.sum()
flat_value = df[df.Account == 'Flat'].Amount.sum()

### On-Going Charges
includes Service Charge and Mortgage interest

In [ ]:
setup_date = datetime.datetime(2017, 6, 1)
now = datetime.datetime.now()
investment_time = (now - setup_date).days/365.25
print(f'We have owned the flat for {investment_time*12:,.0f} months.')

In [ ]:
service_charge_payments = (df.FullType == 'Service Charge') & (df.Amount < 0)
mortgage_interest_payments = (df.FullType == 'Mortgage Interest')
total_ongoing_charges = df[service_charge_payments | mortgage_interest_payments].Amount.sum()
print(f'The on-going charges are: £{-total_ongoing_charges/investment_time/12:,.0f} per momth.')

### Initial Set Up Cost
includes sollicitor fees, stamp duty, and renovation work

In [ ]:
flat_purcahse_fees_payments = df.FullType == 'Flat Purchase Fees'
flat_work_payments = df.FullType == 'FlatWork'
total_setup_cost = df[flat_purcahse_fees_payments | flat_work_payments].Amount.sum()
print(f'The initial setup cost was: £{-total_setup_cost:,.0f} (i.e. {-total_setup_cost/flat_value:.2%} of the flat value).')

### Cost per Month and yearly-return

In [ ]:
total_flat_costs = total_ongoing_charges + total_setup_cost
print(f'The full cost so far was £{-total_flat_costs:,.0f} in total, or £{-total_flat_costs / investment_time / 12:,.0f} per month.')

In [ ]:
expected_rent = 1950
savings_per_year = expected_rent*12 + total_flat_costs/investment_time
print(f'Over the same time period, the total cost of renting would have been: £{expected_rent*12*investment_time:,.0f}')
print(f'In absolute value, the savings over the lifetime of the investment was thus: £{expected_rent*12*investment_time + total_flat_costs:,.0f}')

In [ ]:
initial_asset_allocation = flat_value - df[df.FullType == 'Flat Purchase Capital'].Amount.sum()
current_asset_allocation = flat_value + outstanding_mortgage
average_asset_allocation = 0.5 * (initial_asset_allocation + current_asset_allocation)
print(f'With an average capital allocation of £{average_asset_allocation:,.0f}, the average savings versus ' \
      + f'rents over the lifetime of the investment was: {savings_per_year / average_asset_allocation:.2%} p.a.')